In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

### Extract CSVs into DataFrames

In [4]:
freshwater_file = "Raw Data\CO2_Per_Cap.csv"
#reshwater_df = pd.read_csv(freshwater_file)
#freshwater_df.head()

ParserError: Error tokenizing data. C error: Expected 3 fields in line 5, saw 67


In [5]:
line     = []
expected = []
saw      = []     
cont     = True 

while cont == True:     
    try:
        data = pd.read_csv(freshwater_file,skiprows=line)
        cont = False
    except Exception as e:    
        errortype = e.message.split('.')[0].strip()                                
        if errortype == 'Error tokenizing data':                        
          cerror      = e.message.split(':')[1].strip().replace(',','')
          nums        = [n for n in cerror.split(' ') if str.isdigit(n)]
          expected.append(int(nums[0]))
          saw.append(int(nums[2]))
          line.append(int(nums[1])-1)
        else:
          cerror      = 'Unknown'
           print 'Unknown Error - 222'

if line != []:

IndentationError: unexpected indent (1530578195.py, line 20)

In [3]:
metadata_file = "../Resources/metadata.csv"
metadata_df = pd.read_csv(metadata_file)
metadata_df.head()

,Country Code,Region,IncomeGroup,SpecialNotes,TableName,Unnamed: 5
0,ABW,Latin America & Caribbean,High income,NaN,Aruba,NaN
1,AFE,NaN,NaN,"26 countries, stretching from the Red Sea in t...",Africa Eastern and Southern,NaN
2,AFG,South Asia,Low income,The reporting period for national accounts dat...,Afghanistan,NaN
3,AFW,NaN,NaN,"22 countries, stretching from the westernmost ...",Africa Western and Central,NaN
4,AGO,Sub-Saharan Africa,Lower middle income,NaN,Angola,NaN


### Transform premise DataFrame

In [4]:
# Create a filtered dataframe from specific columns
freshwater_cols = ["Country Name", "Country Code", "2017"]
freshwater_transformed = freshwater_df[freshwater_cols].copy()

# Rename the column headers
freshwater_transformed = freshwater_transformed.rename(columns={"Country Name": "country_name",
                                                          "Country Code": "country_code",
                                                          "2017": "freshwater_bil_m3_2017"})

# Clean the data by dropping duplicates and NA values and setting the index
freshwater_transformed = freshwater_transformed.dropna(how="any")
freshwater_transformed.drop_duplicates("country_code", inplace=True)
freshwater_transformed.set_index("country_code", inplace=True)

freshwater_transformed.head()

,country_name,freshwater_bil_m3_2017
country_code,,
AFE,Africa Eastern and Southern,96.663994
AFG,Afghanistan,20.282000
AFW,Africa Western and Central,30.234050
AGO,Angola,0.705700
ALB,Albania,1.188000


### Transform metadata DataFrame

In [5]:
metadata_cols = ["Country Code", "Region", "IncomeGroup"]
metadata_transformed = metadata_df[metadata_cols].copy()

# Rename the column headers
metadata_transformed = metadata_transformed.rename(columns={"Country Code": "country_code",
                                                         "Region": "region",
                                                         "IncomeGroup": "income_group"})

# Set index
metadata_transformed.set_index("country_code", inplace=True)

metadata_transformed.head()

,region,income_group
country_code,,
ABW,Latin America & Caribbean,High income
AFE,NaN,NaN
AFG,South Asia,Low income
AFW,NaN,NaN
AGO,Sub-Saharan Africa,Lower middle income


### Create database connection

In [6]:
connection_string = "postgres:postgres@localhost:5432/world_db"
engine = create_engine(f'postgresql://{connection_string}')
insp = inspect(engine)

In [7]:
# Confirm tables
insp.get_table_names()

['freshwater', 'metadata']

### Load DataFrames into database

In [8]:
freshwater_transformed.to_sql(name='freshwater', con=engine, if_exists='append', index=True)

In [9]:
metadata_transformed.to_sql(name='metadata', con=engine, if_exists='append', index=True)